## Post-processing

### Settings

In [9]:
import re
import os
import pandas as pd
import numpy as np
from tqdm import trange
from matplotlib import pyplot as plt

cams = ['cam_0_trimmed', 'cam_1_trimmed', 'cam_2_trimmed', 'cam_3_trimmed']
# cams = ['cam_0', 'cam_1', 'cam_2', 'cam_3']
postfix = 'DLC_resnet50_Pop_freeReach_0317_mergedApr19shuffle1_170000_full.pickle'
iteration = 18
session_folder = '1217'
calib_folder = 'calib_1217'

os_path = '/home/myp7435/Pop_freeReach_0317_merged-Min-2020-04-19/videos/'
calib_path = os.path.join(os_path, calib_folder)
pickle_folder =  os.path.join(os_path, session_folder, 'iteration-'+str(iteration))
pickle_paths = [os.path.join(pickle_folder, cam+postfix) for cam in cams]
paths_to_save_raw_csv = [os.path.join(pickle_folder, cam+'.csv') for cam in cams]

joints = ['Wrist', 'CMC_thumb', 'MCP_thumb', 'MCP1', 'MCP2', 'MCP3', 'MCP4',
          'IP_thumb', 'PIP1', 'PIP2', 'PIP3', 'PIP4', 'Dip1', 'Dip2', 'Dip3', 'Dip4',
          'Tip_thumb', 'Tip1', 'Tip2', 'Tip3', 'Tip4']

fingers = [[
  ["Wrist", "CMC_thumb"],
  ["CMC_thumb", "MCP_thumb"],
  ["MCP_thumb", "IP_thumb"],
  ["IP_thumb", "Tip_thumb"]],
    
  [["Wrist", "MCP1"],
  ["MCP1", "PIP1"],
  ["PIP1", "Dip1"],
  ["Dip1", "Tip1"]],
    
  [["Wrist", "MCP2"],
  ["MCP2", "PIP2"],
  ["PIP2", "Dip2"],
  ["Dip2", "Tip2"]],

  [["Wrist", "MCP3"],
  ["MCP3", "PIP3"],
  ["PIP3", "Dip3"],
  ["Dip3", "Tip3"]],
    
  [["Wrist", "MCP4"],
  ["MCP4", "PIP4"],
  ["PIP4", "Dip4"],
  ["Dip4", "Tip4"]]]

# Measured connection length in mm
dist_meds_real = {'Wrist-CMC_thumb': [23.77, 20.83, 23.26],
                  'CMC_thumb-MCP_thumb': [18.73, 24.96, 22.56],
                  'MCP_thumb-IP_thumb': [14.25, 18.23, 17.72],
                  'IP_thumb-Tip_thumb': [11.66, 9.87, 10.53],
                  'Wrist-MCP1': [43.86, 42.38, 35.63],
                  'Wrist-MCP2': [42.81, 39.57, 35.35],
                  'Wrist-MCP3': [42.81, 37.46, 36.32],
                  'Wrist-MCP4': [40.50, 40.22, 36.89],
                  'MCP1-PIP1': [26.32, 27.40, 25.19],
                  'MCP2-PIP2': [30.07, 33.44, 30.00],
                  'MCP3-PIP3': [30.27, 32.70, 28.53],
                  'MCP4-PIP4': [21.95, 24.52, 24.44],
                  'PIP1-Dip1': [18.01, 16.92, 13.58],
                  'PIP2-Dip2': [23.08, 20.38, 18.19],
                  'PIP3-Dip3': [21.98, 21.16, 20.16],
                  'PIP4-Dip4': [18.29, 18.25, 14.78],
                  'Dip1-Tip1': [11.54, 12.82, 11.55],
                  'Dip2-Tip2': [16.18, 15.39, 12.98],
                  'Dip3-Tip3': [14.72, 13.34, 14.15],
                  'Dip4-Tip4': [11.17, 11.64, 10.57]}

### Convert pickle to csv

In [10]:
from utils.data_utils import convertPickleToCSV

convertPickleToCSV(postfix, joints, pickle_paths, paths_to_save_raw_csv)

100%|██████████| 3300/3300 [00:42<00:00, 76.98it/s]


### Create labeled videos

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
import os
os.environ['DLClight']='True'
import deeplabcut

project_name = 'Groot_freeReach-Ben-2021-04-01/'

home_dir = '/home/myp7435/'
path_config_file = os.path.join(home_dir, project_name, 'config-Copy2.yaml')

video_names = ['cam_0.avi','cam_1.avi','cam_2.avi','cam_3.avi']
date = '0218'
iteration = 'iteration-1'
vidfolder = os.path.join(home_dir, project_name, 'videos', date)

videofile_path = []
for vid in video_names:
    videofile_path.append(os.path.join(vidfolder, vid))
    
destfolder = os.path.join(vidfolder, iteration)

 deeplabcut.create_labeled_video(path_config_file, videofile_path, videotype="mp4", destfolder=destfolder)

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


### Interpolate 2D

In [4]:
from utils.data_utils import interpDroppedData2

csv_paths = [os.path.join(pickle_folder, cam+'.csv') for cam in cams]
timestamp_paths = [os.path.join(os_path, session_folder, cam+'_logfile.txt') for cam in cams]
paths_to_save_interp_csv = [os.path.join(pickle_folder, cam+'_interp.csv') for cam in cams]

interpDroppedData2(postfix, joints, csv_paths, timestamp_paths, paths_to_save_interp_csv)

[19244.59143877 19244.56236362 19244.74103451 19244.56236362]


### Load config file

In [1]:
from utils.utils import load_config
from calibration.intrinsic import calibrate_intrinsic
from calibration.extrinsic import calibrate_extrinsic

config = load_config('config_pop.toml')
# config['paths_to_2d_data'] = paths_to_save_interp_csv
# config['calibration']['calib_video_path'] = calib_path
# config['output_video_path'] = pickle_folder
# config['triangulation']['reconstruction_output_path'] = pickle_folder

### Calibration

In [2]:
calibrate_intrinsic(config)
calibrate_extrinsic(config)

  0%|                                | 2/1800 [00:00<01:31, 19.60it/s]


calibrating...

intrinsics_0.toml already exists.

intrinsics_1.toml already exists.

intrinsics_2.toml already exists.

intrinsics_3.toml already exists.

calibration took 0 minutes and 0.0 seconds


100%|█████████████████████████████| 1800/1800 [00:35<00:00, 50.17it/s]


[(('0', '1'), 72), (('0', '2'), 388), (('0', '3'), 1197), (('1', '0'), 72), (('1', '2'), 33), (('1', '3'), 2), (('2', '0'), 388), (('2', '1'), 33), (('2', '3'), 393), (('3', '0'), 1197), (('3', '1'), 2), (('3', '2'), 393)]

Before bundle adjustment, mean reprojection error is 0.09986
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.5811e-03                                    7.27e-01    
       1              2         7.6782e-04      8.13e-04       1.83e+02       1.18e-01    
       2              3         7.6404e-04      3.78e-06       2.52e+00       3.90e-05    
       3              4         7.6404e-04      4.10e-11       4.21e-02       4.51e-06    
`ftol` termination condition is satisfied.
Function evaluations 4, initial cost 1.5811e-03, final cost 7.6404e-04, first-order optimality 4.51e-06.

bundle adjustment took 7.7 seconds

After bundle adjustment, mean reprojection error is 0.07509


### 3D reconstruction

In [2]:
from triangulation.triangulate import reconstruct_3d_ransac
recovery = reconstruct_3d_ransac(config, 2, model_type='hrnet')

100%|█████████████████████████| 69300/69300 [00:29<00:00, 2316.11it/s]


### Outlier removal

In [9]:
from post_processing.outlier_removal import outlier_speed_removal, \
    outlier_connection_removal

from post_processing.filtering import filter_3d

# Representative speed threshold for points of interest in m/s
# Speed of  PIP:  0.75 m/s
#           DIP:  0.9  m/s
#           Tip:  1.2  m/s
#           rest: 0.6  m/s
speed_thr = [0.6, 0.75, 0.9, 1.2]
# This function removes any outlier points that passed over the speed threshold
# This function will save "output_3d_data_speed.csv" inside "pickle_folder"
outlier_speed_removal(config, joints, speed_thr)

# This function 1) removes any connection that is outside of (0.6*med, 1.4*med) range,
#               2) and then (med-2*std, med+2*std)
# This function will save "output_3d_data_out1.csv" and "output_3d_data_out2.csv"
outlier_connection_removal(config, fingers)

# This function perfrom 1) interpolation, 2) filter, and 3) remove outlier connection
# There are two kinds of interpolation methods: linear ('linear') and cubic spline ('spline')
# Interpolate at most 4 missing consecutive data
# There are three kinds of filter you can use: lpf, savgol, None
# lpf: 8 Hz zero-phase low pass filter (output_3d_data_lpf.csv)
# savgol: Savitzky–Golay filter with 5th order and window size of 7 (output_3d_data_savgol.csv)
# None: Doesn't apply filter, it just perform interpolation (output_3d_data_None.csv)
interp_type = 'linear'
filt_type = 'lpf'
filter_3d(config, joints, fingers, interp_type, filt_type)

/home/myp7435/post_processing/outlier_removal.py:35: RuntimeWarning: invalid value encountered in greater
  above_thr = np.squeeze(np.argwhere(joint_speed > y) + 1)
/home/myp7435/post_processing/outlier_removal.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_speed[joint+'_x'][above_thr] = np.nan
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/myp7435/post_processing/outlier_removal.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

***

## Frame extraction steps

### First extract images and manually delete repetitive images

In [ ]:
from utils.vis_utils import extract_frames

img_format = 'png'

# Path to folder where videos are located
vidfolder = '/media/minyoungpark/T7 Touch/pop_1217/'

vidnames = ['cam_0.avi']
vidpaths = [os.path.join(vidfolder, vidname) for vidname in vidnames]

# Path to parent folder where you want to save extracted images
output_folder = '/media/minyoungpark/T7 Touch/for_raquel/'
# Full path to save images
ext = '0218'
folders = [vid.split('.')[0] + '_' + ext for vid in vidnames]
paths_to_save_imgs1 = [os.path.join(output_folder, folder) for folder in folders]

# Manually find time intervals (in seconds) to extract and label
times = [(11, 14),
         (16, 18),
         (20, 22),
         (24, 26),
         (28, 32),
         (36, 39),
         (43, 45),
         (67, 69),
         (73, 75),
         (103, 107)]
every_n_frames = 5
for vidpath, path_to_save in zip(vidpaths, paths_to_save_imgs1):
    extract_frames(vidpath, times, every_n_frames, path_to_save, img_format)

### Extract images with corresponding frame counts after deleting repetitive images

In [ ]:
from utils.vis_utils import extract_specific_frames

vidnames = ['cam_1.avi',
            'cam_2.avi', 'cam_3.avi']
vidpaths = [os.path.join(vidfolder, vidname) for vidname in vidnames]

# Full path to save images
folders = [vid.split('.')[0] + '_' + ext for vid in vidnames]
paths_to_save_imgs2 = [os.path.join(output_folder, folder) for folder in folders]

# Read remaining images after you manually deleted repetitive images
images_folder = paths_to_save_imgs1[0]
image_indices = [int(re.findall(r'\d+', file)[0]) 
                 for file in os.listdir(images_folder) if file.endswith('.'+img_format)]


for vidpath, path_to_save in zip(vidpaths, paths_to_save_imgs2):
    extract_specific_frames(vidpath, image_indices, path_to_save, img_format)